In [1]:
from sahi.models.ultralytics import UltralyticsDetectionModel
from sahi.utils.cv import read_image
from sahi.utils.file import download_from_url
from sahi.predict import get_prediction, get_sliced_prediction, predict
from sahi.scripts.coco_evaluation import evaluate
from IPython.display import Image
import fiftyone as fo

### SAHI

SAHI is a method to enhance inference of small object in following way: image is split into slices which full cover image, small slices overlap each other. Each slice is then passed to a model, after all slices are processes, conflicting detections in overlaping areas are resolved. In this notebook, model is passed to SAHI an dperformed sliced inference. result is then converted to `fiftyone` format, which can perform validation:

In [2]:
NANO = 'gala/train3/'
SMALL = 'gala/train2/'
NANO_SLICED = 'gala_sliced/train3/'

MODELDIR = NANO_SLICED
device = 'mps'

model = MODELDIR + 'weights/best.pt'
image_size = 640 if MODELDIR == NANO_SLICED else 736

data = 'datasets/new/images/val/'

model_type = 'yolov11'
confidence_threshold=0.3
slice_height=640
slice_width=640
overlap_height_ratio=0.15
overlap_width_ratio=0.15

detection_model = UltralyticsDetectionModel(
    model_path=model,
    confidence_threshold=confidence_threshold,
    device=device,
    image_size=image_size
)

/Users/jenda/unive/.venv/lib/python3.13/site-packages/sahi/utils/torch.py:89: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  elif mps and getattr(torch, "has_mps", False) and has_torch_mps:  # prefer MPS if available


In [3]:
dataset = fo.Dataset.from_dir(dataset_dir = 'datasets/new/', dataset_type=fo.types.YOLOv5Dataset)

You are running the oldest supported major version of MongoDB. Please refer to https://deprecation.voxel51.com for deprecation notices. You can suppress this exception by setting your `database_validation` config parameter to `False`. See https://docs.voxel51.com/user_guide/config.html#configuring-a-mongodb-connection for more information


09/17/2025 13:09:47 - WARNING - fiftyone.core.odm.database -   You are running the oldest supported major version of MongoDB. Please refer to https://deprecation.voxel51.com for deprecation notices. You can suppress this exception by setting your `database_validation` config parameter to `False`. See https://docs.voxel51.com/user_guide/config.html#configuring-a-mongodb-connection for more information


 100% |█████████████████| 127/127 [92.2ms elapsed, 0s remaining, 1.4K samples/s]   


09/17/2025 13:09:47 - INFO - eta.core.utils -    100% |█████████████████| 127/127 [92.2ms elapsed, 0s remaining, 1.4K samples/s]   


In [55]:
for sample in dataset:
    pred = get_sliced_prediction(
        sample.filepath,
        detection_model,
        slice_height = slice_height,
        slice_width = slice_width,
        overlap_height_ratio = overlap_height_ratio,
        overlap_width_ratio = overlap_width_ratio
    )
    detections =  pred.to_fiftyone_detections()
    sample['predictions'] = fo.Detections(detections = detections)
    sample.save()

Performing prediction on 6 slices.
Performing prediction on 6 slices.
Performing prediction on 6 slices.
Performing prediction on 6 slices.
Performing prediction on 6 slices.
Performing prediction on 6 slices.
Performing prediction on 8 slices.
Performing prediction on 6 slices.
Performing prediction on 6 slices.
Performing prediction on 8 slices.
Performing prediction on 6 slices.
Performing prediction on 8 slices.
Performing prediction on 6 slices.
Performing prediction on 6 slices.
Performing prediction on 6 slices.
Performing prediction on 6 slices.
Performing prediction on 6 slices.
Performing prediction on 6 slices.
Performing prediction on 6 slices.
Performing prediction on 6 slices.
Performing prediction on 6 slices.
Performing prediction on 6 slices.
Performing prediction on 8 slices.
Performing prediction on 6 slices.
Performing prediction on 6 slices.
Performing prediction on 6 slices.
Performing prediction on 6 slices.
Performing prediction on 6 slices.
Performing predictio

In [57]:
results = dataset.evaluate_detections(pred_field='predictions', gt_field='ground_truth', compute_mAP=True,)
results.print_report()

Evaluating detections...


09/16/2025 23:24:54 - INFO - fiftyone.utils.eval.detection -   Evaluating detections...


 100% |█████████████████| 127/127 [136.3ms elapsed, 0s remaining, 931.5 samples/s]     


09/16/2025 23:24:54 - INFO - eta.core.utils -    100% |█████████████████| 127/127 [136.3ms elapsed, 0s remaining, 931.5 samples/s]     


Performing IoU sweep...


09/16/2025 23:24:54 - INFO - fiftyone.utils.eval.coco -   Performing IoU sweep...


 100% |█████████████████| 127/127 [156.6ms elapsed, 0s remaining, 811.2 samples/s]     


09/16/2025 23:24:54 - INFO - eta.core.utils -    100% |█████████████████| 127/127 [156.6ms elapsed, 0s remaining, 811.2 samples/s]     


              precision    recall  f1-score   support

        ball       0.69      0.78      0.74       128

   micro avg       0.69      0.78      0.74       128
   macro avg       0.69      0.78      0.74       128
weighted avg       0.69      0.78      0.74       128



In [58]:
predict(
    model_type=model_type,
    model_path=model,
    model_device=device,
    model_confidence_threshold=confidence_threshold,
    source='videos/clip_mobil/',
    slice_height=slice_height,
    slice_width=slice_width,
    overlap_height_ratio=overlap_height_ratio,
    overlap_width_ratio=overlap_width_ratio,
    export_crop=True
)

/Users/jenda/unive/.venv/lib/python3.13/site-packages/sahi/utils/torch.py:89: UserWarning:

'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'



There are 231 listed files in folder: clip_mobil/


Performing inference on images:   0%|                                                                 | 0/231 [00:00<?, ?it/s]

Performing prediction on 8 slices.


Performing inference on images:   0%|▏                                                        | 1/231 [00:00<02:56,  1.30it/s]

...

Prediction time is: 345.48 ms
Performing prediction on 8 slices.


Performing inference on images: 100%|██████████████████████████████████████████████████████▊| 230/231 [01:40<00:00,  2.37it/s]

Prediction time is: 336.07 ms
Performing prediction on 8 slices.


Performing inference on images: 100%|███████████████████████████████████████████████████████| 231/231 [01:40<00:00,  2.30it/s]

Prediction time is: 353.51 ms
Prediction results are successfully exported to runs/predict/exp16
